In [15]:
import cobra
from cobra.io import load_model, read_sbml_model
from cobra.flux_analysis import flux_variability_analysis
import pandas as pd
import scipy
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
import importlib
import gc
from cobra.sampling import sample
from matplotlib.ticker import MaxNLocator

In [ ]:
organism="Ecoli_iJO1366"
BOUND = 1000

In [ ]:
model = load_model("iJO1366")
wt_biomass = model.reactions.get_by_id("BIOMASS_Ec_iJO1366_WT_53p95M")
model.objective = wt_biomass
# Update gene reaction rule so it uses reaction IDs 
for reaction in model.reactions:
    reaction.gene_reaction_rule = reaction.id

In [ ]:
# Crear archivo de medio por defecto  
model.reactions.EX_glc__D_e.lower_bound=-1.46222
solution=model.optimize()
with open('medium.csv', 'w', newline="\n") as f:
    print("Metabolite_Name", "Reaction_ID", sep="\t", file=f)
    for reaction in model.summary().uptake_flux.index:
        metabolite = reaction.split("_")[1]
        print(metabolite, reaction, sep="\t", file=f)

In [ ]:
for reaction in model.reactions:
    if reaction.lower_bound <0: reaction.lower_bound=-BOUND
    if reaction.lower_bound >0: reaction.lower_bound=0
    if reaction.upper_bound <0: reaction.upper_bound=0
    if reaction.upper_bound >0: reaction.upper_bound=BOUND

In [ ]:
cobra.io.write_sbml_model(model, "ecoli_iJO1366.xml")

In [27]:
s = pd.read_csv('../figs2_3/ecoli_iJO1366/sampled_flux_distributions.csv',index_col=0)

In [30]:
import pandas as pd
import os # 1. Import the 'os' module for directory and path operations

# 2. Define the names of the new directories
known_fluxes_dir = 'knownFluxes'
transcriptomes_dir = 'transcriptomes'
inputFile_dir = "inputFiles"

# Create the directories before the loop starts.
# 'exist_ok=True' prevents an error if the directories already exist.
os.makedirs(known_fluxes_dir, exist_ok=True)
os.makedirs(transcriptomes_dir, exist_ok=True)
os.makedirs(inputFile_dir, exist_ok=True)
print(f"Ensured directories '{known_fluxes_dir}' and '{transcriptomes_dir}' exist.")


# Create the knownFluxes file for the current iteration
known_fluxes_filename = os.path.join(known_fluxes_dir, f'knownFluxes_common.csv')
with open(known_fluxes_filename, 'w', newline="\n") as f:
    print("Reaction_ID", "Metabolite_ID", "Reaction_Flux", sep="\t", file=f)
    intakeRxns = {"EX_glc__D_e":-1.46222,"BIOMASS_Ec_iJO1366_WT_53p95M":0.1}
    for Reaction_ID,Reaction_Flux in intakeRxns.items():
        Metabolite_ID = Reaction_ID.removeprefix('EX_')
        print(Reaction_ID, Metabolite_ID, Reaction_Flux, sep="\t", file=f)

# Now, start the loop to process each row and append to the file.
for i in range(s.shape[0]):
    
    fluxesRef = s.loc[i]
    expressions = fluxesRef.abs()

    # --- Create iteration-specific files inside their respective folders ---
    
    # Create the transcriptome file for the current iteration
    transcriptome_filename = os.path.join(transcriptomes_dir, f'transcriptome_{i}.csv')
    with open(transcriptome_filename, 'w', newline="\n") as f:
        print("Gene_ID", "Expression", sep="\t", file=f)
        for reactionID in expressions.index:
            print(reactionID, expressions[reactionID], sep="\t", file=f)

    inputFile_filename     = os.path.join(inputFile_dir, f"inputData_{i}.csv")
    with open(inputFile_filename, 'w', newline="\n") as input_data_file:
        # Write the header to the main file just one time.
        print("Organism", "Condition", "GeneExpFile", "Medium", "Network", "KnownFluxes", sep="\t", file=input_data_file)
        Organism = organism
        Condition = f"glucose_iter_{i}"
        GeneExpFile = transcriptome_filename
        Medium = "medium.csv"
        Network = "ecoli_iJO1366.xml"
        KnownFluxes = known_fluxes_filename
        # Print the new line to the already-open input_data_file
        print(Organism, Condition, GeneExpFile, Medium, Network, KnownFluxes, sep="\t", file=input_data_file)


Ensured directories 'knownFluxes' and 'transcriptomes' exist.
